In [0]:
%sql

--PERFILAR UN FILTRO SOBRE LA TABLA RAW PARA LOS REQUEST_NUMBERS "SET" EN VEZ DE "BX.universal_request IS NOT NULL"

/*SELECT DISTINCT 
AX.*,
BX.universal_request,
CX.stage_key,
DX.stage_key AS old_stage_key
FROM bronze_layer.brnz_state_updates AX
LEFT JOIN bronze_layer.brnz_task BX
ON AX.`set` = BX.number
LEFT JOIN silver_layer.slvr_stage CX
on AX.new = CX.stage_name
LEFT JOIN silver_layer.slvr_stage DX
on AX.old = DX.stage_name
WHERE BX.universal_request IS NOT NULL
*/
DELETE FROM silver_layer.slvr_request_stage;
MERGE INTO silver_layer.slvr_request_stage target
USING (
WITH BASE AS  (
SELECT DISTINCT 
AX.*,
BX.universal_request,
CX.stage_key,
DX.stage_key AS old_stage_key
FROM bronze_layer.brnz_state_updates AX
LEFT JOIN bronze_layer.brnz_task BX
ON AX.`set` = BX.number
LEFT JOIN silver_layer.slvr_stage CX
on AX.new = CX.stage_name
LEFT JOIN silver_layer.slvr_stage DX
on AX.old = DX.stage_name
WHERE BX.universal_request IS NOT NULL
)
SELECT 
CONCAT(universal_request,'|', `update`) AS integration_id,
universal_request AS universal_request_pk,
 `update` AS update_pk,
stage_key,
old_stage_key,
to_timestamp(update_time, 'dd/MM/yyyy H:mm') AS update_time_ts,
user_name,
`set` AS request_number,  
current_timestamp() AS created_at_ts, 
current_timestamp() AS modified_at_ts, 
'CSV' AS data_source
FROM BASE
) source
ON target.integration_id = source.integration_id
WHEN NOT MATCHED
  THEN INSERT (integration_id,universal_request_pk, update_pk,stage_key,old_stage_key,update_time_ts,user_name,request_number, created_at_ts,modified_at_ts,data_source)
  VALUES(
  source.integration_id,
  source.universal_request_pk, 
  source.update_pk,
  source.stage_key,
  source.old_stage_key,
  source.update_time_ts,
  source.user_name,
  source.request_number, 
  source.created_at_ts,
  source.modified_at_ts,
  source.data_source);

SELECT * FROM silver_layer.slvr_request_stage;

In [0]:
%sql



MERGE INTO silver_layer.slvr_request_comment target
USING (
SELECT
CONCAT(universal_request,'|',request_comment_id) AS integration_id,
universal_request AS universal_request_pk,
request_comment_id AS request_comment_id,
`timestamp` AS created_date_ts,
`owner`,
`comment`,
current_timestamp() AS created_at_ts, 
current_timestamp() AS modified_at_ts, 
'CSV' AS data_source
FROM bronze_layer.brnz_request_comment 
) source
ON target.integration_id = source.integration_id
WHEN NOT MATCHED
  THEN INSERT (integration_id,universal_request_pk, request_comment_id,created_date_ts,`owner`,`comment`, created_at_ts,modified_at_ts,data_source)
  VALUES(
  source.integration_id,
  source.universal_request_pk, 
  source.request_comment_id,
  source.created_date_ts,
  source.`owner`,
  source.`comment`, 
  source.created_at_ts,
  source.modified_at_ts,
  source.data_source);

SELECT * FROM silver_layer.slvr_request_comment;

In [0]:
%sql
DROP TABLE IF EXISTS silver_layer.slvr_service_request_raw;

CREATE TABLE silver_layer.slvr_service_request_raw AS
WITH TASK_MAX_UPDATE AS (
    SELECT
        AX.universal_request,
        max(AX.sys_updated_on) as max_update 
    FROM bronze_layer.brnz_task AX
    WHERE
        AX.universal_request IS NOT NULL AND
        AX.state != "Cancelled"
    GROUP BY universal_request
),
TASK_FILTERED AS (
    SELECT 
        AX.number, 
        AX.universal_request,
        AX.sys_created_on,
        AX.sys_created_by, 
        AX.assigned_to,
        AX.assignment_group,
        CASE WHEN AX.state = "Closed" THEN "Closed"
         --WHEN AX.state = "Cancelled" THEN "Closed"
         WHEN AX.state = "Resolved" THEN "Closed"
         ELSE "Open" END AS status,
         AX.state,
        AX.sys_updated_on,
        AX.closed_at, 
        AX.service_offering,
        AX.company,
        CASE 
            WHEN AX.sys_class_name in ("Auction Service","Procurement Case") THEN "Procurement"
            WHEN `ref_sc_req_item.cat_item.sc_catalogs` = "TAMM" THEN "TAMM"
            WHEN `ref_sc_req_item.cat_item.sc_catalogs` = "GovDigital" THEN "GovDigital"
            ELSE AX.sys_class_name 
        END AS vertical_name
    FROM bronze_layer.brnz_task AX
    INNER JOIN TASK_MAX_UPDATE BX
    ON AX.universal_request = BX.universal_request
    AND AX.sys_updated_on = BX.max_update
    WHERE AX.state != "Cancelled"
),
TASK_SLA_FILTERED AS (
    SELECT
        `task.universal_request` AS universal_request,
        business_percentage / 100 AS business_percentage, 
        planned_end_time,
        has_breached,
        start_time,
        end_time
    FROM (
      SELECT
      *
      FROM (
    SELECT
        AX.*,
        ROW_NUMBER() OVER (PARTITION BY `task.universal_request` ORDER BY sys_updated_on ASC) AS order_update  
        FROM bronze_layer.brnz_task_sla AX
        )
     WHERE order_update = 1
     )
    WHERE
        planned_end_time IS NOT NULL
),
CSAT_MAX_UPDATED AS (
    SELECT
        AX.`instance.task_id` AS universal_request,
        max(AX.sys_updated_on) as max_update 
    FROM bronze_layer.brnz_csat AX
    WHERE
        AX.`instance.task_id` IS NOT NULL
    GROUP BY AX.`instance.task_id`
),
CSAT_FILTERED AS (
    SELECT
        AX.`instance.task_id` AS universal_request,
        AX.actual_value AS score,
        ((AX.actual_value - 1) / 4) * 0.75 + 0.25 AS percentage_satisfaction
    FROM bronze_layer.brnz_csat AX
    INNER JOIN CSAT_MAX_UPDATED BX
    ON AX.`instance.task_id` = BX.universal_request
    AND AX.sys_updated_on = BX.max_update
    WHERE AX.actual_value != -1
),
SERVICE_REQUEST_TRANS_1 AS (
SELECT
    AX.universal_request,
    AX.number AS request_number,
    AX.sys_created_on AS created_date_ts,
    AX.sys_created_by AS opened_by, 
    AX.assigned_to,
    AX.assignment_group,
    AX.state,
    AX.sys_updated_on AS updated_date_ts,
    AX.closed_at AS closed_date_ts, 
    AX.service_offering AS service,
    AX.company AS entity,
    AX.vertical_name AS vertical,
    CASE WHEN AX.status = "Open" THEN
      CASE WHEN BX.business_percentage >= 1 OR BX.has_breached = "true" THEN "Breached"
           WHEN BX.business_percentage < 0.7 AND BX.business_percentage >= 0 THEN "Within SLA"
           WHEN BX.business_percentage >= 0.7 AND BX.business_percentage <1 THEN "At risk" END
        WHEN AX.status = "Closed" THEN
      CASE WHEN BX.business_percentage >=1 OR BX.has_breached = "true" THEN "Breached"
      ELSE "Within SLA" END
    END AS sla_status,
    AX.status,
    BX.business_percentage, 
    BX.planned_end_time AS sla_planned_end_date_ts,
    BX.has_breached,
    BX.start_time AS sla_start_date_ts,
    BX.end_time AS sla_end_date_ts,
    CX.score,
    CX.percentage_satisfaction
FROM TASK_FILTERED AX
INNER JOIN TASK_SLA_FILTERED BX
ON AX.universal_request = BX.universal_request
LEFT JOIN CSAT_FILTERED CX
ON AX.universal_request = CX.universal_request
--WHERE BX.planned_end_time IS NOT NULL
)
SELECT
universal_request,
request_number,
created_date_ts,
opened_by, 
assigned_to,
assignment_group,
state,
updated_date_ts,
closed_date_ts, 
service,
entity,
vertical,
status,
sla_status,
"No data" AS complaints,
business_percentage, 
sla_planned_end_date_ts,
has_breached,
sla_start_date_ts,
sla_end_date_ts,
score,
percentage_satisfaction,
CASE WHEN sla_status = "Breached" THEN
     CASE WHEN status = "Open" THEN DATEDIFF(CURRENT_DATE, sla_end_date_ts)
          WHEN status = "Closed" THEN DATEDIFF(closed_date_ts, sla_end_date_ts) END
END AS breached_sla_days,
CASE WHEN status = "Open" AND sla_status in ("Within SLA","At Risk") THEN DATEDIFF(sla_end_date_ts, CURRENT_DATE)
END AS remaining_days
FROM SERVICE_REQUEST_TRANS_1
;


In [0]:
%sql

DELETE FROM silver_layer.slvr_fact_service_request;

MERGE INTO silver_layer.slvr_fact_service_request target 
USING (
SELECT
AX.universal_request AS integration_id,
AX.universal_request AS universal_request_pk,
AX.created_date_ts,
COALESCE(BX.entity_key, -1) AS entity_key,
COALESCE(CX.vertical_key, -1) AS vertical_key,
COALESCE(DX.service_key, -1) AS service_key,
current_timestamp() AS created_at_ts, 
current_timestamp() AS modified_at_ts, 
'CSV' AS data_source
FROM silver_layer.slvr_service_request_raw AX
LEFT JOIN silver_layer.slvr_entity BX
ON AX.entity = BX.entity_name
LEFT JOIN silver_layer.slvr_vertical CX
ON AX.vertical = CX.vertical_name
LEFT JOIN silver_layer.slvr_service DX
ON AX.service = DX.service_name
) source
ON target.integration_id = source.universal_request_pk
WHEN NOT MATCHED
THEN INSERT (
integration_id,
universal_request_pk,
created_date_ts,
entity_key,
vertical_key,
service_key,
created_at_ts,
modified_at_ts,
data_source
)
VALUES(
source.integration_id,
source.universal_request_pk,
source.created_date_ts,
source.entity_key,
source.vertical_key,
source.service_key,
source.created_at_ts,
source.modified_at_ts,
source.data_source);

SELECT * FROM silver_layer.slvr_fact_service_request;

In [0]:
%sql

